# Pore type prediction from thin-section images 1.0

In this notebook we explore the usage of thin-section patches to predict whether the central pixel of these patches are part of a moldic pore or not. This first attempt uses an Encoder Neural Network (EncoderNN from pre_sal_ii.models.nn) that has fully connected layers in which it reduces the amount of data in each layer until there is only 1 number in the output. I also tried a Reduction Convolutional Neural Network, but that didn't work, I suppose because it requires a lot of data to really converge.

This notebook is in constant evolution, first I started with a simple model without validation, then I implemented cross-validation, which involved the creation of training classes and functions. These were moved to libraries for general usage in this project. To support K-folded cross validations on images, I also developed methods to partition the image in K parts that can be used as folded datasets instead of randomly distributing patches of the image between all folds. These were also moved to libraries.

In [ ]:
import os
print(os.getcwd())

In [ ]:
from pre_sal_ii.improc import colorspace

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Extracting pores from image

In [ ]:
from importlib import reload
from pre_sal_ii.improc import scale_image_and_save, adjust_gamma

import pre_sal_ii.models as models
reload(models)
models.set_all_seeds(0)

In [ ]:
image_name = "ML-tste_original"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

image_name = "ML-tste_classidicada"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

In [ ]:
image_name = "ML-tste_original"
path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage: np.ndarray = cv2.imread(path)
inputImage = adjust_gamma(inputImage, 0.5)
plt.imshow(inputImage[:,:,::-1])
print(f"inputImage shape: {inputImage.shape}")

# BGR to CMKY:
inputImageCMYK = colorspace.bgr2cmyk(inputImage)

In [ ]:
(C, M, Y, K) = (inputImageCMYK[..., 0],
                inputImageCMYK[..., 1],
                inputImageCMYK[..., 2],
                inputImageCMYK[..., 3])

In [ ]:
binaryImage = cv2.inRange(
    inputImageCMYK,
    (92,   0,   0,   0),
    (255, 255,  64, 196))
binaryImage

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_ERODE, kernel, iterations=1)
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_DILATE, kernel, iterations=1)
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_DILATE, kernel, iterations=1)
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_ERODE, kernel, iterations=1)

plt.imshow(binaryImage, cmap='gray')
cv2.imwrite("../out/some.jpg", binaryImage)
porosidade = np.sum(binaryImage/255)/binaryImage.size
print(f"porosidade = {porosidade}")

In [ ]:
from skimage.measure import label, regionprops

label_img = label(binaryImage)
regions = regionprops(label_img)

In [ ]:
all_objs = []
for it, region in enumerate(regions):
    ys = (region.coords.T[0] - label_img.shape[0]/2)/(label_img.shape[0]/2)
    xs = (region.coords.T[1] - label_img.shape[1]/2)/(label_img.shape[1]/2)
    obj = {
        "area": region.area,
        "max-dist": max((ys**2 + xs**2)**0.5),
    }
    all_objs.append(obj)

df = pd.DataFrame(all_objs)

In [ ]:
max_dist = max(df["max-dist"])
pores_image3 = np.zeros(label_img.shape, dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8:
        color_value = 255
        pores_image3[region.coords.T[0], region.coords.T[1]] = color_value

print(pores_image3.shape)
plt.imshow(pores_image3, cmap='gray')
cv2.imwrite("../out/binary_image3.jpg", pores_image3)

In [ ]:
max_dist = max(df["max-dist"])
print(label_img.shape)
colored_image3 = np.zeros_like(inputImage, dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8:
        color_value = np.random.randint(0, 255, size=3)
        colored_image3[region.coords.T[0], region.coords.T[1]] = color_value

In [ ]:
plt.imshow(colored_image3)
cv2.imwrite("../out/colored_regions_rem_dist.jpg", colored_image3[:,:,::-1])

## Loading manually categorized image

In [ ]:
image_name = "ML-tste_classidicada"
path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage_cl = cv2.imread(path)
plt.imshow(inputImage_cl[:,:,::-1])

In [ ]:
binaryImage_clRed: np.ndarray = cv2.inRange(
    inputImage_cl,
    #  B,   G,   R
    (  0,   0, 240),
    (  5,   5, 255))
plt.imshow(binaryImage_clRed, cmap='gray')

In [ ]:
cv2.imwrite("../out/binaryImage_clRed.jpg", binaryImage_clRed)

In [ ]:
from skimage.measure import label, regionprops

label_img = label(binaryImage_clRed)
regions = regionprops(label_img)

In [ ]:
all_objs = []
for it, region in enumerate(regions):
    ys = (region.coords.T[0] - label_img.shape[0]/2)/(label_img.shape[0]/2)
    xs = (region.coords.T[1] - label_img.shape[1]/2)/(label_img.shape[1]/2)
    obj = {
        "area": region.area,
        "max-dist": max((ys**2 + xs**2)**0.5),
    }
    all_objs.append(obj)

df = pd.DataFrame(all_objs)

In [ ]:
max_dist = max(df["max-dist"])
binaryImage_clRed_mx = np.zeros(label_img.shape, dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8:
        color_value = 255
        binaryImage_clRed_mx[region.coords.T[0], region.coords.T[1]] = color_value

print(binaryImage_clRed_mx.shape)
plt.imshow(binaryImage_clRed_mx, cmap='gray')

## Partitioning images

Here we use partitioning algorithms to partition training images in folds which are more or less equally distributed in measured area. Each algorithm has its own properties, and the one I liked the most was the K-Means Constrained Model. I also testes Split Regions Model, which just divides the image in rows containing approximately the same amount of pixels, and K-Means Model, which does not have the constraint of distributing areas more or less equally.

In [ ]:
from pre_sal_ii.training.image_clustering import cluster_pixels_kmeans_model
from pre_sal_ii.training.image_clustering import cluster_pixels_kmeans_regions
cp_model2 = cluster_pixels_kmeans_model(binaryImage_clRed_mx, n_regions=8)

In [ ]:
regions2 = cluster_pixels_kmeans_regions(binaryImage_clRed_mx, cp_model2)
regions_color11 = cv2.applyColorMap((regions2 * 30).astype(np.uint8), cv2.COLORMAP_JET)

regions2 = cluster_pixels_kmeans_regions(binaryImage_clRed, cp_model2)
regions_color12 = cv2.applyColorMap((regions2 * 30).astype(np.uint8), cv2.COLORMAP_JET)

regions2 = cluster_pixels_kmeans_regions(pores_image3, cp_model2)
regions_color13 = cv2.applyColorMap((regions2 * 30).astype(np.uint8), cv2.COLORMAP_JET)

In [ ]:
regions_color13.shape

In [ ]:
from pre_sal_ii.training.image_clustering import cluster_pixels_h_splits_model
from pre_sal_ii.training.image_clustering import cluster_pixels_h_splits_regions
splits_mdl = cluster_pixels_h_splits_model(binaryImage_clRed_mx)

In [ ]:
regions3 = cluster_pixels_h_splits_regions(binaryImage_clRed_mx, splits_mdl)
regions_color21 = cv2.applyColorMap((regions3 * 30).astype(np.uint8), cv2.COLORMAP_JET)

regions3 = cluster_pixels_h_splits_regions(binaryImage_clRed, splits_mdl)
regions_color22 = cv2.applyColorMap((regions3 * 30).astype(np.uint8), cv2.COLORMAP_JET)

regions3 = cluster_pixels_h_splits_regions(pores_image3, splits_mdl)
regions_color23 = cv2.applyColorMap((regions3 * 30).astype(np.uint8), cv2.COLORMAP_JET)

In [ ]:
import pickle
from pathlib import Path
from pre_sal_ii.training.image_clustering import cluster_pixels_kmeans_constrained_model
from pre_sal_ii.training.image_clustering import cluster_pixels_kmeans_constrained_regions

cache_path = Path("../models/kmc_model_1.pkl")
cache_path.parent.mkdir(exist_ok=True)

if cache_path.exists():
    # Load cached model
    with open(cache_path, "rb") as f:
        kmc_model = pickle.load(f)
    print("Loaded cached model from disk.")
else:
    # Train the model
    kmc_model = cluster_pixels_kmeans_constrained_model(binaryImage_clRed_mx, fraction=10)
    # Save to cache
    with open(cache_path, "wb") as f:
        pickle.dump(kmc_model, f)
    print("Saved trained model to cache.")


In [ ]:
regions3 = cluster_pixels_kmeans_constrained_regions(binaryImage_clRed_mx, kmc_model)
regions_color31 = cv2.applyColorMap((regions3 * 30).astype(np.uint8), cv2.COLORMAP_JET)

regions3 = cluster_pixels_kmeans_constrained_regions(binaryImage_clRed, kmc_model)
regions_color32 = cv2.applyColorMap((regions3 * 30).astype(np.uint8), cv2.COLORMAP_JET)

regions3 = cluster_pixels_kmeans_constrained_regions(pores_image3, kmc_model)
regions_color33 = cv2.applyColorMap((regions3 * 30).astype(np.uint8), cv2.COLORMAP_JET)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 9))

plt.subplot(3, 3, 1)
plt.imshow(regions_color11[:,:,::-1])
plt.subplot(3, 3, 2)
plt.imshow(regions_color21[:,:,::-1])
plt.subplot(3, 3, 3)
plt.imshow(regions_color31[:,:,::-1])

plt.subplot(3, 3, 4)
plt.imshow(regions_color12[:,:,::-1])
plt.subplot(3, 3, 5)
plt.imshow(regions_color22[:,:,::-1])
plt.subplot(3, 3, 6)
plt.imshow(regions_color32[:,:,::-1])

plt.subplot(3, 3, 7)
plt.imshow(regions_color13[:,:,::-1])
plt.subplot(3, 3, 8)
plt.imshow(regions_color23[:,:,::-1])
plt.subplot(3, 3, 9)
plt.imshow(regions_color33[:,:,::-1])

## Extracting features and targets

In [ ]:
from importlib import reload
import pre_sal_ii.models.nn as nn_models
import pre_sal_ii.models.ds as ds_models
reload(ds_models)

models.set_all_seeds(42)

In [ ]:
import numpy as np
from pre_sal_ii.improc import generate_region_map_from_centroids

centroids = kmc_model.cluster_centers_
regions4 = generate_region_map_from_centroids(np.ones_like(pores_image3), centroids)

prob_base = pores_image3 / 255.0
num_regions = 8

from tqdm import tqdm
prob_masks = []
for i in tqdm(range(num_regions)):
    mask_i = (regions4 == i).astype(float)
    prob_masks.append(prob_base * mask_i)


In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 6))
for i in range(num_regions):
    axes[i // 4, i % 4].imshow(prob_masks[i], cmap='gray')

In [ ]:
from torch.utils.data import DataLoader
fold_count = 8
batch_size = 128
num_samples = int(10000/(fold_count - 1)//batch_size*batch_size)

print(f"num_samples = {num_samples}")
print(f"batch_size = {batch_size}")
print(f"fold_count = {fold_count}")

if False:
    datasets = [
            ds_models.WhitePixelRegionDataset(
                    prob_map, inputImage/255., binaryImage_clRed/255.,
                    num_samples=num_samples, seed=42, use_img_to_tensor=True
                ) for prob_map in prob_masks
            ]
else:
    datasets = [
        ds_models.ProbabilityMapPixelRegionDataset(
                prob_map, inputImage/255., binaryImage_clRed/255.,
                num_samples=num_samples,
                region_size=101, target_region_size=1, seed=4290
            ) for prob_map in prob_masks
        ]

In [ ]:
print(f"get_whites_in_target = {sum([np.array(ds.get_whites_in_target()) for ds in datasets])}")
dataiter = iter(datasets[0])
inputs = next(dataiter)
(img, imgTarget, centerPixel) = inputs
print(f"len(data) = {len(inputs)}")
print(f"min = {torch.min(img)}", f"max = {torch.max(img)}")

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
for it, inputs in enumerate(datasets[0]):
    img = inputs[0]
    imgTarget = inputs[1]
    if it >= 10: break
    # print(img.shape, imgTarget.shape)
    img = img.permute(1,2,0)
    assert img.shape == (101, 101, 3)
    assert imgTarget.shape == (1, 1, 1)
    axes[it//5*2+0, it%5].imshow(img.numpy()[:,:,::-1])
    axes[it//5*2+1, it%5].imshow(imgTarget.numpy(), cmap="gray", vmin=0, vmax=1)

## Setting up training

In this section, we will setup the required objects for training:
- device: cuda or cpu, preferring cuda if available
- models: one model for each fold, all of the same class
- criterion: loss function, which is used to get a vector with the direction of better values in parameter space
- optimizers: one optimizer for each fold, they tell how to navigate the parameter space
- trainer: one trainer for each fold, trainers are responsible for the training process of all epochs of a given fold

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import torch.optim as optim
import torch.nn as nn

models = [nn_models.EncoderNN().to(device) for _ in range(fold_count)]
criterion = nn.MSELoss()
optimizers = [optim.AdamW(models[it].parameters(),
                        lr=1e-4,
                        weight_decay=1e-5,
                       ) for it in range(fold_count)]
models[0]

In [ ]:
import torch.nn.functional as F
from pre_sal_ii.training import Trainer

do_asserts = False

class MyTrainer(Trainer):
    def train_epoch_step(self, inputs):
        imgs = inputs[0].to(self.device)
        if do_asserts: assert (*imgs.shape[1:],) == (3, 101, 101)
        imgs = F.interpolate(
            imgs, size=(32, 32), mode='bilinear',
            align_corners=False)
        if do_asserts: assert (*imgs.shape[1:],) == (3, 32, 32)
        imgs = imgs.reshape(-1, 3*32*32)
        if do_asserts: assert (*imgs.shape[1:],) == (3*32*32,)
        outputs = self.model(imgs)
        return imgs.shape[0], outputs

    def train_epoch_loss(self, inputs, outputs):
        expected = inputs[1].to(self.device)
        expected = torch.squeeze(expected, 1)
        expected = torch.squeeze(expected, 2)
        if do_asserts: assert (*expected.shape[1:],) == (1,)
        loss = self.criterion(outputs, expected, **self.criterion_kwargs)
        return loss

In [ ]:
from pre_sal_ii.training import cross_validate
trainers = [MyTrainer(models[fold], optimizers[fold], criterion, device=device) for fold in range(fold_count)]
print("Training...")
best_models, best_losses = cross_validate(trainers, datasets)

## Persisting and reloading the model

In [ ]:
torch.save({
    "models": [m.state_dict() for m in best_models],
    "fold_losses": best_losses,
}, "../models/supervised-8-folds-1.0.pt")

In [ ]:
models2 = [nn_models.EncoderNN().to(device) for _ in range(fold_count)]
checkpoint = torch.load("../models/supervised-8-folds-1.0.pt")
for i, m in enumerate(models2):
    m.load_state_dict(checkpoint["models"][i])
fold_losses2 = checkpoint["fold_losses"]

## Using the best model to infer using whole image

In this section, we will select all pixels which are within pores of the image. This means extracting a patch around each of these pixels and using the model to infer the output pixel classification as a chance of it being part of a moldic pore. We then render all of the pixel predictions in a single image and save it to later usage. We also combine the prediction with the ground-truth in a single image using green chanel for the ground-truth, and red chanel for the predictions. The correct predictions will appear in yellow (i.e. green + red).

In [ ]:
model = models2[np.argmin(fold_losses2)]

In [ ]:
dataset2 = ds_models.WhitePixelRegionDataset(
    pores_image3, inputImage/255., binaryImage_clRed/255.,
    num_samples=-1, seed=None, use_img_to_tensor=True)
dataloader2 = DataLoader(dataset2, batch_size=1024, shuffle=False)

In [ ]:
trainer_best = MyTrainer(model, None, None, device=device)

In [ ]:
pred_image = np.zeros_like(binaryImage_clRed, dtype=np.uint8)

count_gt_half = 0

with torch.no_grad():
    for it, inputs in enumerate(tqdm(dataloader2)):
        _, _, coords = inputs
        step, outputs = trainer_best.train_epoch_step(inputs)
        Y = outputs

        xs = coords[:,1].cpu().numpy()
        ys = coords[:,0].cpu().numpy()
        vs = Y[:,0].cpu().numpy()
        pred_image[ys, xs] = vs*255

In [ ]:
plt.imshow(pred_image, vmin=0, vmax=255, cmap="gray")
cv2.imwrite("../out/sup_pred_8fold_1.0.jpg", pred_image)


In [ ]:
image_pred_true = np.zeros([*binaryImage_clRed.shape, 3], dtype=np.uint8)
image_pred_true = torch.tensor(image_pred_true, dtype=torch.uint8).permute(2, 0, 1)
image_pred_true[1,:,:] = torch.tensor(binaryImage_clRed, dtype=torch.uint8)
image_pred_true[2,:,:] = torch.tensor(pred_image, dtype=torch.uint8)
image_pred_true = image_pred_true.permute(1, 2, 0)
image_pred_true = image_pred_true.numpy()
plt.imshow(image_pred_true[:,:,::-1])
cv2.imwrite("../out/image_pred_8fold_true1.0.jpg", image_pred_true)
